<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Deep Learning Para Aplicações de Inteligência Artificial com Python e C++</font>
## <font color='blue'>Projeto 1</font>
## <font color='blue'>Implementando o Mecanismo de Atenção em Python Seu Uso de Frameworks</font>

In [1]:
!pip install -q -U watermark

In [2]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Anatomia da Arquitetura Transformer (Com Uso do Framework PyTorch)

Attention Is All You Need: https://arxiv.org/abs/1706.03762

In [3]:
#!pip install -q -U torch

In [4]:
# Imports
import torch
from torch import nn
import warnings
warnings.filterwarnings('ignore')

A classe abaixo implementa um modelo Transformer para aprendizado de sequências usando o PyTorch.

In [5]:
# Classe
class Transformer(nn.Module):

  # Método construtor
  def __init__(self, vocab_size, embedding_dim, n_heads, n_layers, dropout):
    
    # Inicializa o construtor da classe mãe (nn..Module)
    super().__init__()

    # Inicializa atributos
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.n_heads = n_heads
    self.n_layers = n_layers
    self.dropout = dropout

    # Define a camada de embedding que transformará a sequência de entrada em uma sequência de vetores 
    # de dimensão fixa
    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    # Define o mecanismo de auto-atenção multi-headed
    self.attention = nn.MultiheadAttention(embedding_dim, n_heads, dropout = dropout)

    # Define a rede neural feed-forward que será usada para gerar a sequência de saída a partir 
    # da sequência de entrada
    self.feed_forward = nn.Sequential(
        nn.Linear(embedding_dim, embedding_dim),
        nn.ReLU(),
        nn.Linear(embedding_dim, embedding_dim)
    )

    # Define a camada de saída final que transformará a sequência de saída na forma de saída desejada
    self.out = nn.Linear(embedding_dim, vocab_size)

  # Método forward
  def forward(self, x):
    
    # Aplica a camada de embedding à sequência de entrada
    x = self.embedding(x)

    # Aplica o mecanismo multi-headed self-attention 
    x = self.attention(x)

    # Aplica o método feed-forward
    x = self.feed_forward(x)

    # Aplica a camada final
    x = self.out(x)

    return x

Para criar um modelo transformador, você pode instanciar a classe Transformer e especificar os hiperparâmetros do modelo, como o tamanho do vocabulário, a dimensão da embedding, o número de "cabeças" de atenção, o número de camadas e a taxa de dropout. 

In [6]:
# Por exemplo:
modelo = Transformer(vocab_size = 1000, 
                     embedding_dim = 32, 
                     n_heads = 4, 
                     n_layers = 2, 
                     dropout = 0.5)

In [7]:
modelo.modules

<bound method Module.modules of Transformer(
  (embedding): Embedding(1000, 32)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
  )
  (feed_forward): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (out): Linear(in_features=32, out_features=1000, bias=True)
)>

In [8]:
modelo.attention

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
)

## Construindo Um Modelo Transformer (Sem Uso do Framework)

Vamos criar um modelo capaz de prever sequências de comprimento igual a 10 tokens.

Um modelo Transformer consiste em várias partes principais:

- 1- Camada de Embedding: Transforma as palavras em vetores numéricos de tamanho fixo.
- 2- Mecanismo de Atenção: Permite que o modelo foque em diferentes partes da entrada.
- 3- Camadas Encoder e Decoder: Processam os dados sequencialmente.
- 4- Camada Linear e Softmax: Para predições finais.

Para este projeto o grande objetivo é implementar o item 2, mas para deixar o exemplo funcional implementaremos também os itens 1 e 4.

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

### Hiperparâmetros Iniciais

In [9]:
# Imports
import numpy as np

In [10]:
# Dimensão do modelo
dim_model = 64

In [11]:
# Comprimento da sequência
seq_length = 10

In [12]:
# Tamanho do vocabulário
vocab_size = 100

### Camada Embedding

A função embedding é utilizada para converter entradas sequenciais em vetores densos de tamanho fixo. Esses vetores são conhecidos como embeddings e são uma parte fundamental, em especial dos modelos de PLN. 

Esses embeddings são fundamentais para modelos de aprendizado profundo em PLN, pois fornecem uma representação rica e densa de palavras ou tokens, capturando informações contextuais e semânticas que são essenciais para tarefas como tradução automática, classificação de texto, entre outras.

In [13]:
# Define a função para criar uma matriz de embedding
def embedding(input, vocab_size, dim_model):
    
    # Cria uma matriz de embedding onde cada linha representa um token do vocabulário
    # A matriz é inicializada com valores aleatórios normalmente distribuídos
    embed = np.random.randn(vocab_size, dim_model)
    
    # Para cada índice de token no input, seleciona o embedding correspondente da matriz
    # Retorna um array de embeddings correspondentes à sequência de entrada
    return np.array([embed[i] for i in input])

### Mecanismo de Atenção

Neste exemplo, vamos simplificar e usar apenas uma camada de atenção.

Leia o item "Projeto 1 - Componentes Q (Query), K (Key) e V (Value)" no Capítulo 5 com os detalhes sobre os valores Q, K e V.

No Transformer, Q, K e V são derivados da mesma entrada em camadas de atenção do encoder, mas de entradas diferentes no decoder (Q vem da saída da camada anterior do decoder, enquanto K e V vêm da saída do encoder). O mecanismo de atenção calcula um conjunto de pontuações (usando o produto escalar entre Q e K, daí o nome "scaled dot-product attention"), aplica uma função softmax para obter pesos de atenção e usa esses pesos para ponderar os values, criando uma saída que é uma combinação ponderada das informações relevantes de entrada.

Este processo permite que o modelo dê "atenção" às partes mais relevantes da entrada para cada parte da saída, o que é especialmente útil em tarefas como tradução, onde a relevância de diferentes palavras da entrada pode variar dependendo da parte da frase que está sendo traduzida

### Função de Ativação Softmax

A função softmax é uma função de ativação amplamente utilizada em redes neurais, especialmente em cenários de classificação, onde é importante transformar valores brutos de saída (logits) em probabilidades que somam 1. Abaixo, está o código da função softmax com comentários em cada linha explicando seu funcionamento:

In [14]:
# Função de ativação softmax
def softmax(x):
    
    # Calcula o exponencial de cada elemento do input, ajustado pelo máximo valor no input 
    # para evitar overflow numérico
    e_x = np.exp(x - np.max(x))
    
    # Divide cada exponencial pelo somatório dos exponenciais ao longo do último eixo (axis=-1)
    # O reshape(-1, 1) garante que a divisão seja realizada corretamente em um contexto multidimensional
    return e_x / e_x.sum(axis=-1).reshape(-1, 1)

### Scale Dot Product

A função scaled_dot_product_attention() é um componente do mecanismo de atenção em modelos Transformer. Ela calcula a atenção entre conjuntos de queries (Q), keys (K) e values (V). 

Essencialmente, essa função permite que o modelo dê importância diferenciada a diferentes partes da entrada, um aspecto chave que torna os modelos Transformer particularmente eficazes para tarefas de PLN e outras tarefas sequenciais.

In [15]:
# Define a função para calcular a atenção escalada por produto escalar
def scaled_dot_product_attention(Q, K, V):
    
    # Calcula o produto escalar entre Q e a transposta de K
    matmul_qk = np.dot(Q, K.T)
    
    # Obtém a dimensão dos vetores de chave
    depth = K.shape[-1]
    
    # Escala os logits dividindo-os pela raiz quadrada da profundidade
    logits = matmul_qk / np.sqrt(depth)
    
    # Aplica a função softmax para obter os pesos de atenção
    attention_weights = softmax(logits)
    
    # Multiplica os pesos de atenção pelos valores V para obter a saída final
    output = np.dot(attention_weights, V)
    
    # Retorna a saída ponderada
    return output

### Saída do Modelo com Operação Linear e Softmax

A função linear_and_softmax() é uma combinação de uma camada linear seguida por uma função softmax, comumente usada em modelos de aprendizado profundo, especialmente em tarefas de classificação. 

In [16]:
# Define a função que aplica uma transformação linear seguida de softmax
def linear_and_softmax(input):
    
    # Inicializa uma matriz de pesos com valores aleatórios normalmente distribuídos
    # Esta matriz conecta cada dimensão do modelo (dim_model) a cada palavra do vocabulário (vocab_size)
    weights = np.random.randn(dim_model, vocab_size)
    
    # Realiza a operação linear (produto escalar) entre a entrada e a matriz de pesos
    # O resultado, logits, é um vetor que representa a entrada transformada em um espaço de maior dimensão
    logits = np.dot(input, weights)
    
    # Aplica a função softmax aos logits
    # Isso transforma os logits em um vetor de probabilidades, onde cada elemento soma 1
    return softmax(logits)

### Construindo o Modelo Final

In [17]:
# Função do modelo final
def transformer_model(input):
    
    # Embedding
    embedded_input = embedding(input, vocab_size, dim_model)

    # Mecanismo de Atenção 
    attention_output = scaled_dot_product_attention(embedded_input, embedded_input, embedded_input)
    
    # Camada linear e softmax
    output_probabilities = linear_and_softmax(attention_output)

    # Escolhendo os índices com maior probabilidade
    output_indices = np.argmax(output_probabilities, axis=-1)
    
    return output_indices

## Usando o Modelo Para as Previsões

In [18]:
# Gerando dados aleatórios para a entrada do modelo
input_sequence = np.random.randint(0, vocab_size, seq_length)

In [19]:
print("Sequência de Entrada:", input_sequence)

Sequência de Entrada: [60 18  9 49 79 57 11 59 99 12]


In [20]:
# Fazendo previsões com o modelo
output = transformer_model(input_sequence)

In [21]:
print("Saída do Modelo:", output)

Saída do Modelo: [74 53 39 33 67  5 54 98 39 43]


In [22]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [23]:
#%watermark -v -m

In [24]:
#%watermark --iversions

# Fim